# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |


## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.




In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
text = '안녕하세요~'

response = client.embeddings.create(
    model = 'text-embedding-3-small',
    input = [text]
)

print(response)
print(len(response.data[0].embedding))

CreateEmbeddingResponse(data=[Embedding(embedding=[0.006883355788886547, -0.08132949471473694, -0.006404029205441475, 0.021313322708010674, 0.037833377718925476, -0.03754355013370514, -0.04523507505655289, 0.03580459952354431, -0.04623831808567047, -0.02938384748995304, -0.05408589914441109, 0.003915431443601847, 0.012217259965837002, 0.008639030158519745, -0.0372760184109211, 0.02925008349120617, -0.0442318320274353, 0.019251102581620216, -0.027689483016729355, 0.019429456442594528, 0.06153218448162079, 0.014089979231357574, -0.010690102353692055, -0.01994222402572632, 0.03856908902525902, 0.05578026548027992, 0.05685038864612579, -0.027644895017147064, 0.04447706788778305, -0.05132140964269638, 0.02706524357199669, -0.0386582650244236, -0.014524716883897781, -0.052971187978982925, -0.027310481294989586, 0.004160668235272169, 0.0160630214959383, -0.022394593805074692, -0.025214819237589836, -0.03767731785774231, -0.01445783395320177, -0.005434228572994471, -0.004054770339280367, 0.037

In [ ]:
def texts_to_embedding(texts, model = 'text-embedding-3-small'):
  # 특수문자/개행문자 등을 제거하면 임베딩 품질 높아진다.
  texts = [text.replace('\n', ' ') for text in texts]
  response = client.embeddings.create(
      model = model,
      input = texts
  )
  return [data.embedding for data in response.data]

output = texts_to_embedding(['hello world', 'byebye world'])

import numpy as np
np.array(output).shape

(2, 1536)

## 음식리뷰 유사도 검색
https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

In [ ]:
!gdown 1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD

Downloading...
From: https://drive.google.com/uc?id=1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD
To: /content/fine_food_reviews_1k.csv
100% 439k/439k [00:00<00:00, 108MB/s]


In [ ]:
# 데이터로드
import pandas as pd

df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens
Unnamed: 0,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76


In [ ]:
# Summary + Text -> Combined
df['combined'] = df['Summary'].str.strip() + ';' + df['Text'].str.strip() # strip 좌우 공백 제거
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens,combined
Unnamed: 0,,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33,where does one start...and stop... with a tre...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26,Arrived in pieces;Not pleased at all. When I o...
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242,"It isn't blanc mange, but isn't bad . . .;I'm ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216,These also have SALT and it's not sea salt.;I ...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76,Happy with the product;My dog was suffering wi...


In [23]:
# 임베딩변환
df['embedding'] = texts_to_embedding(df['combined'].tolist())
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens,combined,embedding
Unnamed: 0,,,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33,where does one start...and stop... with a tre...,"[0.028519336134195328, -0.018376275897026062, ..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26,Arrived in pieces;Not pleased at all. When I o...,"[0.013578505255281925, 0.033949751406908035, -..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242,"It isn't blanc mange, but isn't bad . . .;I'm ...","[0.0020295188296586275, 0.004446823615580797, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216,These also have SALT and it's not sea salt.;I ...,"[-0.015502768568694592, 0.01425130944699049, -..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76,Happy with the product;My dog was suffering wi...,"[0.0037726855371147394, -0.06684160232543945, ..."


In [24]:
# embed_df로 변환
embed_df = df[['embedding']]
# embed_df = df['embedding'].to_frame()
embed_df.index = df['combined']
embed_df.head()

,embedding
combined,
where does one start...and stop... with a treat like this;Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.028519336134195328, -0.018376275897026062, ..."
"Arrived in pieces;Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[0.013578505255281925, 0.033949751406908035, -..."
"It isn't blanc mange, but isn't bad . . .;I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[0.0020295188296586275, 0.004446823615580797, ..."
"These also have SALT and it's not sea salt.;I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.015502768568694592, 0.01425130944699049, -..."
Happy with the product;My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[0.0037726855371147394, -0.06684160232543945, ..."


In [27]:
# 유사도 검색
from sklearn.metrics.pairwise import cosine_similarity

def review_search(query, emb_df, top_n=5):
    # 검색어 동일한 임베딩변환
    query_emb = texts_to_embedding([query])

    embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])
    top_n_texts = embed_df.sort_values('cos_sim', ascending=False).head(top_n)
    return top_n_texts

review_search('best coffee', embed_df)

/tmp/ipython-input-27-20207283.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])


,embedding,cos_sim
combined,,
super coffee;Great coffee and so easy to brew. This coffee has great aroma and is good to the last drop. I actually like all the brands. This is the way coffee should taste!!,"[-0.0010457742027938366, -0.039594460278749466...",0.613942
super coffee;Great coffee and so easy to brew. This coffee has great aroma and is good to the last drop. I actually like all the brands. This is the way coffee should taste!!,"[-0.0010457742027938366, -0.039594460278749466...",0.613942
super coffee;Great coffee and so easy to brew. This coffee has great aroma and is good to the last drop. I actually like all the brands. This is the way coffee should taste!!,"[-0.0010457742027938366, -0.039594460278749466...",0.613942
super coffee;Great coffee and so easy to brew. This coffee has great aroma and is good to the last drop. I actually like all the brands. This is the way coffee should taste!!,"[-0.0010457742027938366, -0.039594460278749466...",0.613942
super coffee;Great coffee and so easy to brew. This coffee has great aroma and is good to the last drop. I actually like all the brands. This is the way coffee should taste!!,"[-0.0010457742027938366, -0.039594460278749466...",0.613942


In [28]:
review_search('bad delievery', embed_df)

/tmp/ipython-input-27-20207283.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])


,embedding,cos_sim
combined,,
"great product, poor delivery;The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.030943654477596283, -0.010354330763220787,...",0.466266
"great product, poor delivery;The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.030943654477596283, -0.010354330763220787,...",0.466266
"great product, poor delivery;The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.030943654477596283, -0.010354330763220787,...",0.466266
"great product, poor delivery;The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.030943654477596283, -0.010354330763220787,...",0.466266
"The Seller is NOT Starbursts;As I ordered this for my boyfriend for his birthday as he LOVES the sour Starbursts but they were discontinued. So I ordered these and found that not only is the seller a scammer, I never received my product!<br /><br />After multiple attempts and emails to ARRAM, I had to file a claim. They said they would ""redeliver"" this product THREE times and I still didn't receive it. They write their own feedback for a higher score and never solved my problem.<br /><br />DO NOT ORDER THESE! You will never receive them. Very disappointed that I was scammed on Amazon. I never write reviews however, I wanted to ensure that this does not happen to anyone else.","[-0.002867665607482195, -0.020793406292796135,...",0.391809
